# PoC

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

In [3]:
logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

In [4]:
log = logging.getLogger(__name__)

In [5]:
from omegaconf import OmegaConf

In [6]:
c = OmegaConf.load("../config/main.yaml")

In [7]:
c.settings.debug = False
c.wandb.enabled = False
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

c.params.model = "ump_1dcnn"

In [8]:
log.info(OmegaConf.to_yaml(c))

2022-02-01 22:20:07,770 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ${hydra:runtime.cwd}/../cache
  group: default
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
params:
  seed: 440
  n_class: 1
  n_fold: 5
  skip_training: false
  epoch: 20
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: simple_cpcv
  group_name: investment_id
  time_name: time_id
  label_name: target
  feature_set:
  - f000
  dataset: ump_1
  model: ump_1dcnn
  pretrained: []
  criterion: RMSELoss
  optimizer: Adam
  scheduler: CosineAnnealingWarmupRestarts
  lr: 

In [9]:
import torch

In [10]:
import src.utils as utils

In [11]:
device = utils.gpu_settings(c)

2022-02-01 22:20:09,149 [INFO] [utils] CUDA_VISIBLE_DEVICES: 6,7
2022-02-01 22:20:09,205 [INFO] [utils] torch device: cuda, device count: 2


## Fold

In [12]:
from src.load_data import InputData

In [13]:
input = InputData(c)

2022-02-01 22:20:09,472 [INFO] [load_data] Load feather file. path: ../../inputs/train.f
2022-02-01 22:20:37,115 [INFO] [utils] Mem. usage decreased to 366Mb: 49% reduction
2022-02-01 22:20:37,152 [INFO] [load_data] Load feather file. path: ../../inputs/example_test.f
2022-02-01 22:20:37,340 [INFO] [utils] Mem. usage decreased to 0.0Mb: 49% reduction
2022-02-01 22:20:37,341 [INFO] [load_data] Load feather file. path: ../../inputs/example_sample_submission.f
2022-02-01 22:20:37,347 [INFO] [utils] Mem. usage decreased to 0.0Mb: 34% reduction


In [14]:
input.train.fillna({"time_fold": c.params.n_fold}, inplace=True)

In [15]:
input.train

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,group_fold,time_fold
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566,1.0,0.0
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624,3.0,0.0
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608,0.0,0.0
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309,4.0,0.0
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141405,1219_3768,1219,3768,0.033600,0.093530,-0.720275,-0.345497,-0.438781,-0.166972,-0.437182,...,-0.660579,0.875537,0.421628,-0.428097,-0.075548,-0.533092,-0.193732,-0.581394,2.0,5.0
3141406,1219_3769,1219,3769,-0.223264,-1.344935,-0.199987,-0.107702,-0.454677,-0.221914,-0.141174,...,-0.670493,0.875537,0.421628,-0.729949,-1.514277,0.013145,-0.890270,-0.589705,2.0,5.0
3141407,1219_3770,1219,3770,-0.559415,0.979489,-1.110491,1.006980,-0.467307,-0.159549,1.355671,...,0.820784,-1.142157,0.421628,-0.363329,1.363181,-0.079106,-1.580124,-0.297625,2.0,5.0
3141408,1219_3772,1219,3772,0.009599,-2.565332,0.320301,0.076600,1.380182,-0.155366,-0.689000,...,0.133074,-1.142157,0.421628,-0.375288,-1.514277,-0.973762,0.608647,-0.372040,0.0,5.0


In [16]:
input.train["group_fold"].value_counts()

1.0    628285
0.0    628283
4.0    628281
3.0    628281
2.0    628280
Name: group_fold, dtype: int64

In [17]:
input.train["time_fold"].value_counts()

0.0    523570
1.0    523568
5.0    523568
4.0    523568
3.0    523568
2.0    523568
Name: time_fold, dtype: int64

In [18]:
input.train[["group_fold", "time_fold"]].value_counts()

group_fold  time_fold
1.0         0.0          107189
0.0         3.0          106248
1.0         2.0          105776
3.0         1.0          105700
2.0         1.0          105484
4.0         1.0          105467
3.0         3.0          105462
            5.0          105437
            4.0          105423
0.0         4.0          105269
2.0         0.0          105064
4.0         5.0          104911
            0.0          104903
0.0         5.0          104838
            2.0          104803
4.0         2.0          104687
2.0         4.0          104554
            2.0          104495
            5.0          104380
1.0         4.0          104375
4.0         3.0          104366
2.0         3.0          104303
1.0         5.0          104002
0.0         0.0          103962
4.0         4.0          103947
3.0         2.0          103807
1.0         1.0          103754
            3.0          103189
0.0         1.0          103163
3.0         0.0          102452
dtype: int64

In [19]:
from src.make_fold import CPCV_INDEX_5FOLD

In [20]:
fold = 1
df = input.train
len(df[df["time_fold"].isin(CPCV_INDEX_5FOLD["val_time_id"][fold])].index)

1047138

In [21]:
len(
    df[
        (~df["time_fold"].isin(CPCV_INDEX_5FOLD["val_time_id"][fold]))
        & (df["group_fold"] != CPCV_INDEX_5FOLD["val_group_id"][fold])
    ].index
)

1674754

## Model

In [22]:
from src.make_model import make_model

In [23]:
model = make_model(c, device)

In [24]:
model

DataParallel(
  (module): OneDCNNModel(
    (expand): Sequential(
      (0): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=300, out_features=1024, bias=True)
      (2): CELU(alpha=0.06)
    )
    (conv1): Sequential(
      (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): Conv1d(128, 384, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (3): ReLU()
      (4): AdaptiveAvgPool1d(output_size=4)
      (5): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.1, inplace=False)
      (7): Conv1d(384, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (8): ReLU()
    )
    (conv2): Sequential(
      (0): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.3, inplace=False)
      (2): Conv1d(384, 384, kernel_size=(3,), stri

In [25]:
input_ = torch.randn(5, 300).to(device)

In [26]:
model(input_)

tensor([[ 0.4668],
        [ 0.3757],
        [-1.1719],
        [-0.3882],
        [ 0.0988]], device='cuda:0', dtype=torch.float16,
       grad_fn=<GatherBackward>)